In [1]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk import *
from nltk.text import Text
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [33]:
file_path = "ESConv.json"

with open(file_path, "r", encoding="utf-8") as file:
    dataset = json.load(file)

In [34]:
dataset[0]

{'dialog': [{'annotation': {}, 'content': 'Hello\n', 'speaker': 'seeker'},
  {'annotation': {'strategy': 'Question'},
   'content': 'Hello, what would you like to talk about?',
   'speaker': 'supporter'},
  {'annotation': {},
   'content': 'I am having a lot of anxiety about quitting my current job. It is too stressful but pays well\n',
   'speaker': 'seeker'},
  {'annotation': {'strategy': 'Question'},
   'content': 'What makes your job stressful for you?',
   'speaker': 'supporter'},
  {'annotation': {'feedback': '5'},
   'content': 'I have to deal with many people in hard financial situations and it is upsetting \n',
   'speaker': 'seeker'},
  {'annotation': {'strategy': 'Question'},
   'content': 'Do you help your clients to make it to a better financial situation?',
   'speaker': 'supporter'},
  {'annotation': {},
   'content': 'I do, but often they are not going to get back to what they want. Many people are going to lose their home when safeguards are lifted \n',
   'speaker': '

In [42]:
# Extract conversations from the corpus
corpus_data = []
for corpus in dataset:
    dialog = corpus['dialog']
    corpus_data.append(dialog)

df = pd.DataFrame({'Dialog': pd.Series(corpus_data)})

df

,Dialog
0,"[{'speaker': 'seeker', 'annotation': {}, 'cont..."
1,"[{'speaker': 'supporter', 'annotation': {'stra..."
2,"[{'speaker': 'supporter', 'annotation': {'stra..."
3,"[{'speaker': 'supporter', 'annotation': {'stra..."
4,"[{'speaker': 'supporter', 'annotation': {'stra..."
5,"[{'speaker': 'supporter', 'annotation': {'stra..."
6,"[{'speaker': 'supporter', 'annotation': {'stra..."
7,"[{'speaker': 'seeker', 'annotation': {}, 'cont..."
8,"[{'speaker': 'seeker', 'annotation': {}, 'cont..."
9,"[{'speaker': 'supporter', 'annotation': {'stra..."


In [46]:
seeker_content = []
for index, row in df.iterrows():
    if row['Dialog'][0]['speaker'] == 'seeker':
        seeker_content.append(row['Dialog'][0]['content'])

In [47]:
for content in seeker_content:
    print(content)

Hello

Hey there

I need some tips to overcome from that problem
Hi
Hi

I recently lost my job due to Covid 19 and I'm scared.

Hi
Hi
Hi I am feeling blue, things aren't going well at all
Hello
Hello

hello

Hai friend how are you?
Hello there

Hello?

Hello

Hi!

Hi!

I need understanding at this time.
Hello.
hi
Hi there

Hello

Good afternoon
Good morning

Hi

Too Much Work Pressure 
Hello, how are you doing today?

My friends are trying to shame me.
hi

Hello, it's nice to meet you.
Good evening
Hello

Hello.

Well, it has been a hard week.
I still see him from time to time online, but I don't know why he just dropped me.

Hey there, how's it going?

I cannot believe my Girl is breaking up with me....
hello
hello
what ever i study, i am not able to remember the contents which is necessary for my academic tests
Well, I have some bad news.
Hello
Hello. Feeling low today.

I need support to feel ok during the pandemic
Hello.
Hey, how are you doing?

Hi there

Hello. 

Hello.

Hello, I 